# Agents

Agents use an LLM to determine which actions to take and in what order.
An action can either be using a tool and observing its output, or returning to the user.

When used correctly agents can be extremely powerful. The purpose of this notebook is to show you how to easily use agents through the simplest, highest level API. If you want more low level control over various components, check out the documentation for custom agents (coming soon).

## Concepts

In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
- LLM: The language model powering the agent.
- Agent: The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. If you want to implement a custom agent, see the documentation for custom agents (coming soon).

**For a list of supported agents and their specifications, see [here](../explanation/agents.md)**

## Tools
When constructing your own agent, you will need to provide it with a list of Tools that it can use. A Tool is defined as below.

```python
class Tool(NamedTuple):
    """Interface for tools."""

    name: str
    func: Callable[[str], str]
    description: Optional[str] = None
```

The two required components of a Tool are the name and then the tool itself. A tool description is optional, as it is needed for some agents but not all.

## Loading an agent


In [1]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

In [2]:
# Load the tool configs that are needed.
from langchain import LLMMathChain, SerpAPIWrapper
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [3]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [4]:
agent.run("How old is Olivia Wilde's boyfriend? What is that number raised to the 0.23 power?")



> Entering new ZeroShotAgent chain...
How old is Olivia Wilde's boyfriend? What is that number raised to the 0.23 power?
Thought: I need to find out how old Olivia Wilde's boyfriend is, and then use a calculator to calculate the power.
Action: Search
Action Input: Olivia Wilde's boyfriend age
Observation: While Wilde, 37, and Styles, 27, have both kept a low profile when it comes to talking about their relationship, Wilde did address their ...
Thought: Olivia Wilde's boyfriend is 27 years old.
Action: Calculator
Action Input: 27^0.23

> Entering new LLMMathChain chain...
27^0.23

```python
import math
print(math.pow(27, 0.23))
```

Answer: 2.1340945944237553

> Finished LLMMathChain chain.

Observation: Answer: 2.1340945944237553

Thought:
> Finished ZeroShotAgent chain.


'2.1340945944237553'

In [5]:
# We can also return the intermediate steps
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [6]:
agent({"input":"How old is Olivia Wilde's boyfriend? What is that number raised to the 0.23 power?"})



> Entering new ZeroShotAgent chain...
How old is Olivia Wilde's boyfriend? What is that number raised to the 0.23 power?
Thought: I need to find out how old Olivia Wilde's boyfriend is, and then use a calculator to calculate the power.
Action: Search
Action Input: Olivia Wilde's boyfriend age
Observation: While Wilde, 37, and Styles, 27, have both kept a low profile when it comes to talking about their relationship, Wilde did address their ...
Thought: Olivia Wilde's boyfriend is 27 years old.
Action: Calculator
Action Input: 27^0.23

> Entering new LLMMathChain chain...
27^0.23

```python
import math
print(math.pow(27, 0.23))
```

Answer: 2.1340945944237553

> Finished LLMMathChain chain.

Observation: Answer: 2.1340945944237553

Thought:
> Finished ZeroShotAgent chain.


{'input': "How old is Olivia Wilde's boyfriend? What is that number raised to the 0.23 power?",
 'output': '2.1340945944237553',
 'intermediate_steps': [{'log': " I need to find out how old Olivia Wilde's boyfriend is, and then use a calculator to calculate the power.\nAction: Search\nAction Input: Olivia Wilde's boyfriend age",
   'tool': 'Search',
   'tool_input': "Olivia Wilde's boyfriend age",
   'observation': 'While Wilde, 37, and Styles, 27, have both kept a low profile when it comes to talking about their relationship, Wilde did address their ...'},
  {'log': " Olivia Wilde's boyfriend is 27 years old.\nAction: Calculator\nAction Input: 27^0.23",
   'tool': 'Calculator',
   'tool_input': '27^0.23',
   'observation': 'Answer: 2.1340945944237553\n'}]}